# Fundamentos Matemáticos de la Criptografía
En esta Notebook se muestran ejemplos de las operaciones elementales usadas en algunos algoritmos criptográficos

## 1. Aritmética Modular
Se presentan las operaciones elementales usadas en el anillo Zn

In [1]:
# Funciones:
# ----------
# Calcular Potencia modular:
def calcularPotencia(base,exponente,modulo):
    res=(base**exponente)%modulo
    return int(res)
# Factorizar un numero pseudo-primo
def factorizar(numero):
    numero_raiz=int(numero**(0.5))
    res=str(numero)+" es primo"
    for i in range(1,numero_raiz):
        entero=int(numero/i)
        decimal=float(float(numero)/float(i))
        if(entero==decimal):
            res=str(numero)+"="+str(entero)+"*"+str(i)
    return res
# Calcular inverso modular
def inverso(numero,modulo):
    res=0
    for i in range(numero):
        entero=int((i*modulo+1)/numero)
        decimal=float(float(i*modulo+1)/float(numero))
        if(entero==decimal):
            res=entero
    return res
# Listar Primos:
def listarPrimos(limite):
    lista=[]
    res=0
    for i in range(2,limite):
        for j in range(2,i):
            if((i/j)==float(int(i/j))):
                res=str(i)+" no es primo"
                break
        if(res!=str(i)+" no es primo"):
            lista.append(i)
    return lista

In [2]:
# Inicialización
a=23
b=56
modulo=37
num=4819

### Potencia Modular

In [3]:
pot=calcularPotencia(a,b,modulo)
print(str(a)+" exp "+str(b)+" (mod"+str(modulo)+") = "+str(pot))

23 exp 56 (mod37) = 26


### Inverso modular
Notar que si "i" es el inverso modular de "a", entonces: a*i(mod n)=1

In [4]:
i=inverso(a,modulo)
print(str(a)+" (exp) (-1) (mod "+str(modulo)+") = "+str(i))
print("y además: "+str(i)+" * "+str(a)+" (mod "+str(modulo)+") = "+str(a*i%modulo))

23 (exp) (-1) (mod 37) = 29
y además: 29 * 23 (mod 37) = 1


### Factorizar

In [5]:
res=factorizar(num)
print(res)

4819=79*61


### Listar Primos
Dado un número, esta función presenta todos los números primos menores a dicho número

In [6]:
res=listarPrimos(modulo)
print("Los numeros primos menores a "+str(modulo)+" son: "+str(res))

Los numeros primos menores a 37 son: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]


## 2. RSA:
Usaremos las funciones anteriores para implementar (en modo demo) el algoritmo RSA. El mismo posee 3 etapas: generación de claves, encripción y desencripción.

In [7]:
# Librerias:
import random as random
# Hardcodeo:
umbral=80
textoPlano='CASA'

### Generacion de claves:

In [8]:
# Alice elige primos grandes p y q
primos=listarPrimos(umbral)
p=primos[random.randrange(int(len(primos)/2),len(primos))]
q=primos[random.randrange(int(len(primos)/2),len(primos))]
print('p=',p)
print('q=',q)
while(p==q):
    q=primos[random.randrange(int(len(primos)/2),len(primos))]
n=p*q
fi_n=(p-1)*(q-1)
print('phi=',fi_n)
e=primos[random.randrange(int(len(primos)/2),len(primos))]
while(e==p or e==q):
    e=primos[random.randrange(int(len(primos)/2),len(primos))]
d=inverso(e,fi_n)
K_pub=(e,n)
K_priv=(d,n)
print ("GENERACION DE CLAVES:")
print ("K_pub="+str(K_pub)+" - K_priv="+str(K_priv))

p= 71
q= 71
phi= 2520
GENERACION DE CLAVES:
K_pub=(53, 2627) - K_priv=(1997, 2627)


### Encripcion del texto plano

In [9]:
# Cifrado con clave publica:
print ("mensaje: "+str(textoPlano))
codigoPlano=[]
for i in range(len(textoPlano)):
    num=ord(str(textoPlano[i]))
    codigoPlano.append(num)
print ("codigo mensaje: "+str(codigoPlano))
codigoCifrado=[]
for i in range(len(codigoPlano)):
    cifra=int(calcularPotencia(codigoPlano[i],e,n))
    codigoCifrado.append(cifra)
print ("codigo cifrado: "+str(codigoCifrado))
textoCifrado=''
for i in range(len(codigoCifrado)):
    textoCifrado=str(textoCifrado)+chr(codigoCifrado[i]%128)
print ("texto cifrado: "+str(textoCifrado))

mensaje: CASA
codigo mensaje: [67, 65, 83, 65]
codigo cifrado: [1205, 1188, 2216, 1188]
texto cifrado: 5$($


### Desencripción:

In [10]:
# Descifrado con clave privada:
print ("codigo cifrado: "+str(codigoCifrado))
codigoDescifrado=[]
for i in range(len(codigoCifrado)):
    descifrado=int(calcularPotencia(codigoCifrado[i],d,n))
    codigoDescifrado.append(descifrado)
print ("codigo Descifrado: ",str(codigoDescifrado))
textoDescifrado=''
for i in range(len(codigoDescifrado)):
    textoDescifrado=str(textoDescifrado)+chr(codigoDescifrado[i]%128)
print ("texto Descifrado: "+str(textoDescifrado))

codigo cifrado: [1205, 1188, 2216, 1188]
codigo Descifrado:  [67, 65, 83, 65]
texto Descifrado: CASA


### ¿Puede hackearse RSA?
Supongamos que capturamos un mensaje cifrado con RSA y accedemos a la clave pública. RSA se hackea factorizando el valor del módulo y del coeficiente de encripción "e" presentes en la clave pública. Con ello, puede recuperarse la clave privada, y con ello, se desencripta el mensaje.

In [11]:
# recuperación de 'd' (clave privada)
p_=int((factorizar(n).split("=")[1]).split("*")[0])
q_=int((factorizar(n).split("=")[1]).split("*")[1])
fi__n=(p_-1)*(q_-1)
d_=inverso(e,fi__n)
print("La clave privada reconstruida es: K_priv=("+str(d_)+","+str(n)+")")

La clave privada reconstruida es: K_priv=(1997,2627)


In [12]:
# Descifrado del mensaje:
msjCifrado=codigoCifrado
print("el codigo cifrado capturado es: ",msjCifrado)
codigoDescifrado=[]
for i in range(len(codigoCifrado)):
    descifrado=int(calcularPotencia(codigoCifrado[i],d_,n))
    codigoDescifrado.append(descifrado)
textoDescifrado=''
for i in range(len(codigoDescifrado)):
    textoDescifrado=str(textoDescifrado)+chr(codigoDescifrado[i]%128)
print ("texto Descifrado: "+str(textoDescifrado))
print("--Mensaje Hackeado--")

el codigo cifrado capturado es:  [1205, 1188, 2216, 1188]
texto Descifrado: CASA
--Mensaje Hackeado--


## 3. Vernam
Consiste en un algoritmo simetrico que utiliza la funcion booleana "XOR" u "O Exclusiva" bit a bit ya que la misma es una función simétrica y equilibrada (cumpliendo con uno de los postulados de Golomb). Recordando: la funcion XOR devuelve un 1 si los bits son distintos y un 0 si son iguales.

In [13]:
# Funcion XOR entre enteros
def xor_entre_enteros(num1, num2):
    resultado_xor = num1 ^ num2
    return resultado_xor
# Conversion entero - binario con Longitud fija:
def entero_a_binario(numero):
    binario = bin(numero)[2:]
    longitud_actual = len(binario)
    if longitud_actual < 8:
        ceros_faltantes = 8 - longitud_actual
        binario = '0' * ceros_faltantes + binario
    return binario

In [14]:
# Encripción Simétrica
msj=[67, 65, 83, 65]
print("mensaje: ",msj)
msjBinario=[]
for i in range(len(msj)):
    res=entero_a_binario(msj[i])
    msjBinario.append(res)
print("mensaje Binario:         ",msjBinario)
pasw=[80, 65, 83, 87]
pwdBinario=[]
for i in range(len(msj)):
    res=entero_a_binario(pasw[i])
    pwdBinario.append(res)
print("password Binario:        ",pwdBinario)
msjCifrado=[]
for i in range(len(msj)):
    res=xor_entre_enteros(msj[i],pasw[i])
    msjCifrado.append(res)
msjCifradoBinario=[]
for i in range(len(msj)):
    res=entero_a_binario(msjCifrado[i])
    msjCifradoBinario.append(res)
print("mensaje Cifrado Binario: ",msjCifradoBinario)



mensaje:  [67, 65, 83, 65]
mensaje Binario:          ['01000011', '01000001', '01010011', '01000001']
password Binario:         ['01010000', '01000001', '01010011', '01010111']
mensaje Cifrado Binario:  ['00010011', '00000000', '00000000', '00010110']


In [15]:
# Verificacion: Desencripción
msjPlano=[]
for i in range(len(msj)):
    res=xor_entre_enteros(msjCifrado[i],pasw[i])
    msjPlano.append(res)
print("mensaje descifrado: ",msjPlano)

mensaje descifrado:  [67, 65, 83, 65]
